<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2022/Project-Incident-Team2/blob/master/load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# upload mrjob.conf under current directory(/content/mrjob.conf)
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
# Please fill your aws credential information here
credentials = {
    'region_name': 'us-east-1',
    'aws_access_key_id': 'x',
    'aws_secret_access_key': 'x',
    'aws_session_token': 'x'
}

In [ ]:
!pip install boto3
!pip install PyAthena
import boto3, json

In [9]:
session = boto3.session.Session(**credentials)

In [17]:
from pyathena import connect

cursor = connect(aws_access_key_id="x",
                 aws_secret_access_key="x",
                 aws_session_token='x',
                 s3_staging_dir="s3://vandy-bigdata-finalproject/month=10/month=10/part-00004-427c2bcd-90e5-41f1-a1d3-c4c35772a251.c000.snappy.parquet",
                 region_name="us-east-1").cursor()

In [ ]:
cursor.execute("")
print(cursor.description)
print(cursor.fetchall())

In [ ]:
client = boto3.client('athena')

In [ ]:
response = client.create_named_query(
    Name='TestQuery',
    Database='xxx',
    QueryString='SELECT * FROM "xxx"."yyy" limit 10;',
)
print(response)